# Problem

There is a date discrepency for some of the x-series structures

# Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime
import json

In [ ]:
with open("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/20250212_p_to_x_fred/rmsd_csvs/20250217_combined_results_with_data.json") as f:
    problematic = json.load(f)

In [ ]:
frag_dir = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/mpro_fragalysis-04-01-24")
soaks_path = frag_dir / "extra_files" / "mpro_soaks.csv"
cocrystals_path = frag_dir / "extra_files" / "Mpro_cocrystallisation.csv"

In [ ]:
err_msg = problematic["err_msg"]

In [ ]:
err_msg

In [ ]:
missing = [s[:-3] for s in problematic["missing_reference_structures"]]

## so what I had been doing previously was processing the mpro soaks file

In [ ]:
soaks = pd.read_csv(soaks_path)

### process as in `/Users/alexpayne/Scientific_Projects/asapdiscovery-sars-retrospective/science/20240403_multi_pose_docking_v2/20240430_analyze_cross_docking_results/20240503_inputs_analysis/20240510_add_dates.ipynb`

In [ ]:
def date_processor(date_string):
    if type(date_string) == str and not date_string == 'None':
        try:
            return datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")
        except ValueError:
            return datetime.strptime(date_string, "%d/%m/%Y %H:%M")
    else:
        return None
def process_soaks(soaks):
    ddf = soaks.loc[:, ["Sample Name", "Data Collection Date"]]
    ddf['Sanitized_Date'] = ddf["Data Collection Date"].apply(date_processor)
    ddf.columns = ["Structure_Name", "Data_Collection_Date", "Structure_Date"]
    date_dict = ddf.set_index("Structure_Name").to_dict()["Structure_Date"]
    date_dict = {k: str(v) for k, v in date_dict.items() if str(v) != "NaT"}
    frag_to_cmpd_dict = {row["Sample Name"]: row["Compound ID"] for idx, row in soaks.iterrows() if row["Sample Name"] in date_dict}

    return date_dict, frag_to_cmpd_dict

In [ ]:
soaks_date_dict, soaks_cmpd_to_frag_dict = process_soaks(soaks)
with open("soaks_date_dict.json", "w") as f:
    json.dump(soaks_date_dict, f)
with open("soaks_cmpd_to_frag_dict.json", "w") as f:
    json.dump(soaks_cmpd_to_frag_dict, f)

In [ ]:
all(k not in soaks_date_dict for k in missing)

In [ ]:
all(k not in soaks_cmpd_to_frag_dict for k in missing)

In [ ]:
cocrystals = pd.read_csv(cocrystals_path)

In [ ]:
co_date_dict, co_cmpd_to_frag_dict = process_soaks(cocrystals)

In [ ]:
co_set = set(co_date_dict.keys())
so_set = set(soaks_date_dict.keys())

In [ ]:
len(co_set - so_set)

In [ ]:
len(so_set - co_set)

In [ ]:
len(co_set & so_set)

In [ ]:
len(co_set | so_set)

In [ ]:
all(k in co_date_dict for k in missing)

In [ ]:
co_date_dict

In [ ]:
still_missing = [k for k in missing if k not in co_cmpd_to_frag_dict]

In [ ]:
still_missing

In [ ]:
co_cmpd_to_frag_dict

In [ ]:
combined_date_dict = {**soaks_date_dict, **co_date_dict}

In [ ]:
with open("combined_date_dict.json", "w") as f:
    json.dump(combined_date_dict, f)